In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.colors import TwoSlopeNorm
import cartopy.crs as ccrs
from pathlib import Path

# define function for sat processing

def process_sat(ds):
    ds = ds.drop_vars(["time_bnds", "gw"], errors="ignore").sortby('lat')   
    #ds = ds[['TREFHT']]
    ds['time'] = pd.date_range("2022-02-01", freq='1ME', periods=ds.time.shape[0])
    ds['ens'] = np.arange(1,ds.ens.shape[0]+1)
    
    #ds = ds.rename({'tas': 'sat'})
    # Rename variable if it exists
    var_names = ["tas", "T2", "TREFHT", "ts", "TS"]
    for var in var_names:
        if var in ds.data_vars:
            ds = ds.rename({var: 'sat'})
            ds['sat'].attrs['units'] = 'K'
            break  # Exit the loop after renaming the first matching variable
    ds['sat'].attrs['units'] = 'K'

    return ds


# set default fontsize for Plots
plt.rcParams.update({"font.size": 22})
xr.set_options(keep_attrs=True)


In [2]:
### Load model
#select model name
model = "CMAM"
#chose infilepath
infile_path = Path(f"/sto0/data/Intermediate/Hunga_Tonga_byAles/htmip/htmip/Exp1_fixedSST/{model}")

#select target variable 
what = "tas"
#select infiles with and without HTHH
infiles_w = sorted(infile_path.glob(f"{what}*H2Oonly*.nc"))
infiles_wo = sorted(infile_path.glob(f"{what}*NoVolc*.nc"))

#Open datasets for with and without
ds_w = xr.open_mfdataset(
    infiles_w[:],
    parallel=True,
    combine="nested",
    concat_dim=["ens"]).pipe(process_sat)

ds_wo = xr.open_mfdataset(
    infiles_wo[:],
    parallel=True,
    combine="nested",
    concat_dim=["ens"]).pipe(process_sat)

In [3]:
### Load model
#select model name
model = "CMAM"
#chose infilepath
infile_path = Path(f"/sto0/data/Intermediate/Hunga_Tonga_byAles/htmip/htmip/Exp1_fixedSST/{model}")

#select target variable 
what = "tas"
#select infiles with and without HTHH
infiles_w = sorted(infile_path.glob(f"{what}*H2Oonly*.nc"))
infiles_wo = sorted(infile_path.glob(f"{what}*NoVolc*.nc"))

#Open datasets for with and without
ds_w = xr.open_mfdataset(
    infiles_w[:],
    parallel=True,
    combine="nested",
    concat_dim=["ens"]).pipe(process_sat)

ds_wo = xr.open_mfdataset(
    infiles_wo[:],
    parallel=True,
    combine="nested",
    concat_dim=["ens"]).pipe(process_sat)

In [ ]:
for year in range(2022, 2032):
    print(f"Processing year: {year}")

    # Select year of interest
    ds_sel_season_w = ds_w.sat.sel(time=f"{year}").groupby("time.season").mean("time")
    ds_sel_season_wo = ds_wo.sat.sel(time=f"{year}").groupby("time.season").mean("time")

    # Calculate anomaly
    ds_anom = ds_sel_season_w - ds_sel_season_wo

    # Seasonal anomalies
    seasonal_anom = {}
    seasons = ["DJF", "MAM", "JJA", "SON"]
    for season in seasons:
        seasonal_anom[season] = ds_anom.sel(season=season).mean("ens")

    # Select seasonal data with and without perturbation
    seasonal_w = {season: ds_sel_season_w.sel(season=season) for season in seasons}
    seasonal_wo = {season: ds_sel_season_wo.sel(season=season) for season in seasons}

    # Perform t-test for each season
    pv_data = {}
    for season in seasons:
        _, pv_data[season] = stats.ttest_ind(
            seasonal_w[season], seasonal_wo[season], axis=0, equal_var=False
        )

    # Dynamic color scaling
    vmin = min(data.min().values for data in seasonal_anom.values())
    vmax = max(data.max().values for data in seasonal_anom.values())

    if abs(vmin) > abs(vmax):
        vmax = abs(vmin)
    else:
        vmin = -vmax

    divnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

    # Plotting
    fig, axs = plt.subplots(2, 2, figsize=(16, 7), constrained_layout=True,
                            sharey=True, sharex=True, subplot_kw={"projection": ccrs.PlateCarree()})
    axs = axs.flatten()

    model = "CMAM"

    for ax, season in zip(axs, pv_data.keys()):
        sig_lat, sig_lon = np.where(pv_data[season] <= 0.05)

        mesh = ax.pcolormesh(
            seasonal_anom[season]["lon"],
            seasonal_anom[season]["lat"],
            seasonal_anom[season].values,
            shading="auto",
            cmap="coolwarm",
            norm=divnorm,
            transform=ccrs.PlateCarree()
        )

        # Add significant points
        ax.scatter(
            seasonal_anom[season]["lon"][sig_lon],
            seasonal_anom[season]["lat"][sig_lat],
            color="black",
            marker="x",
            s = 5,
            alpha = 0.2
        )

        # Add coastlines and labels
        ax.coastlines()
        ax.set_title(f"Temperature anomaly {model} {season} {year}", fontsize=12)

        gl = ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {"fontsize": 12}
        gl.ylabel_style = {"fontsize": 12}

    # Add a colorbar
    fig.colorbar(mesh, ax=axs, orientation="vertical", fraction=0.05, pad=0.05, label="Temperature [K]", shrink=0.8)
    fig.suptitle(f"Temperature anomaly {model} {year}")

    # Save plot for each year
    plt.savefig(f"{model}_anom_{year}_seas.png")
    plt.close(fig)

Processing year: 2022
Processing year: 2023
Processing year: 2024
Processing year: 2025
Processing year: 2026
Processing year: 2027
Processing year: 2028
Processing year: 2029
Processing year: 2030
Processing year: 2031
